In [1]:
# Clone Repository of InstantCity
!rm -rf ./InstantCity/
!git clone https://github.com/ualsg/InstantCity

Klone nach 'InstantCity'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 166 (delta 6), reused 3 (delta 3), pack-reused 148
Empfange Objekte: 100% (166/166), 11.39 MiB | 16.69 MiB/s, fertig.
Löse Unterschiede auf: 100% (24/24), fertig.


In [2]:
# Change working directory to InstantCity
%cd InstantCity

/hdd/repository/InstantCITY/InstantCity


In [3]:
# What's inside
!ls

data		 images       options	   run_engine.py  util
environment.yml  LICENSE.txt  __pycache__  test.py
fid.py		 models       README.md    train.py


In [5]:
!cat ./README.md


<p align="center">
  <a href="https://ual.sg/">
    <img src="images/logo.jpg" alt="Logo">
  </a>
  <h3 align="center">InstantCITY: Synthesising morphologically accurate geospatial data for urban form analysis, transfer, and quality control</h3>
  <a >
    <img src="images/Pipeline.png" alt="Logo">
  </a>
</p>

This is the official repo of InstantCITY, a Geospatial Data Translation model for urban form analysis, transfer, and quality control.

## Running InstantCITY 

### 1. Install prerequisites

Use `environment.yml` to create a conda environment for GANmapper

  ```sh
  conda env create -f environment.yml
  conda activate IC
  ```

### 2. Download weights
The weights files are available on figshare in the Checkpoints folder.

```https://doi.org/10.6084/m9.figshare.15103128.v1```

Place the `Checkpoints` folder in the repo.
### 3. Prediction
Predictions can be carried out by running the following sample code. The name of the city depends on the name of each dataset.

 ```sh
 python 

In [6]:
!python --version

Python 3.11.3


In [7]:
# !pip install condacolab
# import condacolab
# condacolab.install()
# !conda --version
# !which conda

In [28]:
# install modules
!pip install pathlib
!pip install dominate
!pip install scipy
!pip install torch
!pip install Pillow
!pip install torchvision

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
error: externally-managed-environment

× This environment is externally managed
╰

In [9]:
# Connect to Google Drive to retreive GANmapper_Data
# from google.colab import drive
# drive.mount('/content/drive')
# !ls /content/drive/MyDrive/GANmapper_Data/

In [10]:
# Copy the testing data from GANmapper_Data
# !cp -r /content/drive/MyDrive/GANmapper_Data/checkpoints /content/InstantCity/
# !cp -r /content/drive/MyDrive/GANmapper_Data/datasets /content/InstantCity/

In [18]:
# Unzip GANmapper data
!unzip -q -o ../GANmapper\ Data.zip
# And move it to the parent folder
!cp -r ./GANmapper\ Data/checkpoints .
!cp -r ./GANmapper\ Data/datasets .

In [19]:
# What's inside datasets?
!ls ./datasets
!ls ./datasets/Exp4/

'Data Processing.ipynb'   Exp4
Jakarta  Jakarta17  LA	Paris  Singapore


In [20]:
# What's inside checkpoints?
!ls ./checkpoints/
!ls ./checkpoints/Exp3/
!ls ./checkpoints/Exp3/Singapore/

Exp1  Exp2  Exp3
Jakarta  LA  Singapore
latest_net_D.pth  loss_log.txt	train_opt.txt
latest_net_G.pth  test_opt.txt	web


In [21]:
# Copy Test-dataset
!mkdir ./datasets/Test
print("Copying data...")
!cp -r ./datasets/Exp4/Singapore/Source/* ./datasets/Test/
print("Copy complete.")
!ls ./datasets/Test/

Copying data...
Copy complete.
16


In [22]:
# Copy Test-model
!mkdir ./checkpoints/SG15
print("Copying data...")
!cp -r ./checkpoints/Exp3/Singapore/* ./checkpoints/SG15/
print("Copy complete.")
!ls ./checkpoints/SG15/

Copying data...
Copy complete.
latest_net_D.pth  loss_log.txt	train_opt.txt
latest_net_G.pth  test_opt.txt	web


In [24]:
# Fix mistakes, that are located in the repo
# ------------------------------------------------------------------------------

# Copy modified python-scripts
!cp ../modified_files/train.py ./
!cp ../modified_files/test.py ./
!cp ../modified_files/pix2pixHD_model.py ./models/
!cp ../modified_files/models.py ./models/

# Copy training images to the training directory
!mkdir ./training_data
!cp -r ./datasets/Exp4/Singapore/Source/* ./training_data/train_A/
!cp -r ./datasets/Exp4/Singapore/Target/* ./training_data/train_B/

mkdir: das Verzeichnis „./training_data“ kann nicht angelegt werden: Die Datei existiert bereits


In [25]:
# What's in the test script?
# !cat test.py

In [26]:
# What are the test's parameters?
# !cat options/test_options.py

In [30]:
# Run the test of model
!python train.py --name custom_model --dataroot ./training_data
# Rename folder in /content/InstantCity/datasets/Exp4/Singapore into "train_A" and "train_B", to work with it
# Remember the model must be saved at "/content/InstantCity/checkpoints/SG15/"
# !python test.py --name SG15 --dataroot ./datasets/Test

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: ./training_data
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: custom_model
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 60
niter_decay: 40
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: 1
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use

In [ ]:
!ls /content/InstantCity/fake\\16/51688/
from PIL import Image
im = Image.open("/content/InstantCity/fake\\16/51688/32519.png")
im.show()
!cp /content/InstantCity/fake\\